# Solving Hamilton-Jacobi-Bellman Equations (Deep Galerkin)
#### Frederik Kelbel, Imperial College London

## Dependencies

In [1]:
import torch
import plotly.graph_objects as go
import numpy as np
from operators import div, Δ, D, mdotb, bdotm, mdotm, bdotb, cat
from DGM import DGMPIASolver, DeepPDESolver
from pdes import HBJ, PDE
from scipy.integrate import quad
from plotly.subplots import make_subplots
from configs import CONFIG_HBJS as MODEL_CONFIG
from itertools import product
import torch.nn.functional as F

## Plotting

In [2]:
def plot_losses(losses, avg_over=10):
    avgs_1 = np.convolve(losses[:, 0], np.ones(avg_over), 'valid') / avg_over
    avgs_2 = np.convolve(losses[:, 1], np.ones(avg_over), 'valid') / avg_over
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs_1)), y=avgs_1, mode='lines', name="Value Loss"), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs_2)), y=avgs_2, mode='lines', name="Control Loss"), row=1, col=1)
    fig.update_layout(
        title="Loss",
        xaxis_title="Iterations",
        yaxis_title="Loss",
        font=dict(
            family="Courier New, monospace",
            size=14
        )
    )
    fig.show()
    
def plot_value(solver, sol):
    fig = make_subplots(rows=1, cols=2, 
                   specs=[[{'type': 'surface'}, {'type': 'surface'}]])
    xs = np.linspace(0, 1, 100)
    ts = np.linspace(0.01, 1, 100)
    us_pred = np.array([[solver(x, t).item() for x in xs] for t in ts])
    us = np.array([[sol(x, t) for x in xs] for t in ts])
    fig.add_trace(go.Surface(z=us, showscale=False), row=1, col=1)
    fig.add_trace(go.Surface(z=us_pred), row=1, col=2)
    fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="J(x, t"),
                  scene2 = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="J(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
    fig.show()
    
def plot_loss(losses, avg_over=10):
    avgs = np.convolve(losses, np.ones(avg_over), 'valid') / avg_over
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs)), y=avgs, mode='lines', name="Error at x=0.1"), row=1, col=1)
    fig.update_layout(
        title="Loss",
        xaxis_title="Iterations",
        yaxis_title="Loss",
        font=dict(
            family="Courier New, monospace",
            size=14
        )
    )
    fig.show()

## Problem Formulation

Objective: Find the control process $u = (u_t)_{t \geq 0}$ in admissable set $\mathcal{A}$ for an Itô Process $X^u = (X_t^u)_{t \geq 0}$ satisfying:


$$
d X_t^u = \mu(t, X_t^u, u_t) dt + \sigma(t, X_t^u, u_t) d W_t, \quad X_0^u = 0.
$$

We will consider the HBJ-Equations in their primal form.

The agents performance is assessed via:
$$
J^u(t, x) = \mathbb{E}\Big[ \int_t^T F(s, X_s^u, u_s) ds + G(X_T^u) \;\Big|\; X_t^u = x \Big]
$$

Denote $J(t, x) = \sup_{u \in \mathcal{A}} J^u(t, x)$, then this value function satisfies the following HJB-equations:

$$
\begin{cases}
\partial_t J(t, x) + \sup_{u \in \mathcal{A}} \{\mathscr{L}^u_t J(t, x) + F(t, x, u)\} = 0 \\
J(T, x) = G(x)
\end{cases}
$$

### The Merton Problem (Wealth Allocation Problem)

The goal is to find the optimal wealth allocation strategy over time such that the wealth itself is maximized.

Consider a market with a risky and risk-free asset. Suppose the value of the risk-free asset at time $t$ is given by $\frac{d B_t}{B_t} = r dt$ or $B_t = B_0 e^{rt}, t\geq 0$. Additionally, we have that the risky asset evolves accordingly to $\frac{d S_t}{S_t} = \mu dt + \sigma dW_t$, where $\{W_t\}_{t\geq 0}$ is a standard one-dimensional Brownian motion.

The wealth can then be described by
$$
X_t= x + \int_0^t v_s \frac{dS_s}{S_s} + \int_0^t(X_s-v_s)\frac{dB_s}{B_s},
$$
where $v_t = u_t X_t$ describes the amount of the wealth to be have invested into the risky asset at time $t$. $u_t$ is the fraction of wealth invested in the risky asset.

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$ be a filtered probability space. The evolution of an investor's wealth is described via
$$
\begin{cases}
dX_s = ((\mu -r)u_s + r)X_s ds + \sigma u_s X_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

with $\mu>r$ and $\sigma>0$ referring to drift and volatility, respectively. Let $r>0$ denote the discount rate, i.e. the depreciation constant. The intent is to maximise the objective
$$
J^u(t, X_t) = \mathbb{E}[ X_T^\gamma ], \; \gamma \in(0, 1).
$$

The respective HBJ-equation becomes:

$$
\begin{cases}
\partial_t J(t, x) + \sup_{u} \Big\{ ((\mu-r)u + r)x \partial_x J(t, x) + \frac{1}{2} \sigma^2 u^2 x^2\partial_{xx} J(t, x) \Big\} = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
J(T, x) = x^\gamma \text{ $\forall x > 0$}
\end{cases}
$$

#### Analytical Solution (Oksendal):

Assume $J(t, x) = w(t) v(x)$, with $J(T, x) = w(T) v(x) = x^\gamma$. We guess $J(t, x) = w(t) x^\gamma$, where $w(T)=1$. The problem becomes 
$$
\begin{cases}
w'(t) + \gamma  \sup_{u} \Big\{ (\mu-r) u + r + \frac{1}{2} \sigma^2 u^2 (\gamma-1) \Big\}w(t) = 0
\\
w(T) = x^\gamma
\end{cases}.
$$

Then, $u$ is maximised for $u^* = \frac{\mu-r}{\sigma^2 (1-\gamma)}$ and the equation becomes
$$
\begin{cases}
w'(t) + \frac{\gamma (\mu-r)^2}{\sigma^2(2-2\gamma)}w(t) = 0
\\
w(T) = x^\gamma
\end{cases}.
$$
Thus, we have  $w'(t) = \frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}w(t)$. It follows that $w(t) = \exp\big(\frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}t\big)$. The final solution is:
$$
J(t, x) = w(t)v(x) = \exp\big(\frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}t\big) x^\gamma.
$$

We need to verify this using the HBJ-verification theorem.

In [ ]:
class RISKY_ASSET(HBJ):
    def __init__(self):
        super().__init__()
        self.μ = 0.04
        self.σ = 0.4
        self.r = 0.03
        self.γ = 0.8
        
        self.var_dim_J = 2 # (x, t)
        self.control_vars = [1] # (t)
        self.cost_function = lambda u, var: 0
        self.differential_operator = lambda J, u, var: ((self.μ-self.r)*u+self.r)*var[0]*div(J, var[0]) + 0.5*self.σ**2*u**2*var[0]**2*Δ(J, var[0])
        self.domain_func = [(lambda var: var, 128)]
        self.boundary_cond_J = [lambda J, var: J - var[0]**self.γ]
        self.boundary_func_J = [(lambda var: [var[0], 0*var[1] + 1], 128)]
        self.boundary_cond_u = [lambda u, var: torch.clamp(u, min=1) - torch.clamp(u, max=-1) - 2]
        self.boundary_func_u = [(lambda var: var, 64)]

In [ ]:
eq = RISKY_ASSET()
model = MODEL_CONFIG
solver = DGMPIASolver(model, eq)
loss = np.array(list(solver.train(800)))
plot_losses(loss)

In [ ]:
u_sol = lambda t : (eq.μ-eq.r)/(eq.σ**2*(1-eq.γ))
J_sol = lambda x, t: x**eq.γ * np.exp((eq.γ*(eq.μ-eq.r)**2)/(eq.σ**2*(2*eq.γ-2))*t)

In [ ]:
fig = make_subplots(rows=1, cols=1)
eval_points = np.linspace(0, 1, 100)
fig.add_trace(go.Scatter(x=eval_points, y=[solver.u(p)[0] for p in eval_points], mode='lines', name="Optimal Control"), row=1, col=1)
fig.add_trace(go.Scatter(x=eval_points, y=[u_sol(p) for p in eval_points], mode='lines', name="Optimal Control Solution"), row=1, col=1)
fig.update_layout(
    title="Solutions",
    xaxis_title="Time",
    yaxis_title="Control Signal",
    yaxis_range=[0, 1],
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

In [ ]:
plot_value(solver.J, J_sol)

### Optimal Liquidation Problem

Let us consider an asset $S_t$, that evolves accordingly with Brownian motion, i.e. $dS_s = -B u_s ds + \sigma dW_s$. Assume that we hold a certain amount of this asset and intend to liquidate it over the time period $s \in [0, T]$. Let $Q$ be our inventory of the asset with $dQ_s = - u_s ds$ (with $Q(0)=N$), where $u_s$ represents the liquidation rate. Suppose the temporary price impact is linear (the cost of selling), then our cash $X_s$ is modelled by 
$$
X_t = x + \int_0^t S_s u_s ds -\int_0^t D u_s^2 + C q^2ds 
$$

We aim to maximise
$$
J^u(t, S, q) := \mathbb{E}^{t, S} \Big[ \int_t^T S_s u_s -Du_s^2 - Cq^2 ds + S_T q_T- R q_T^2\Big],
$$
subject to the process
$$\begin{eqnarray}
dS_s &=& -Bu_s ds + \sigma dW_s \\ 
dQ_s &=& - u_s ds
\end{eqnarray}$$

Thus, the HBJ-Equation is given as:
$$
\begin{cases}
\partial_t J(t, S, q) + \sup_{u} \Big\{ u_s(S-Du_s) - Cq^2 + ( \frac{1}{2} \sigma^2 \partial_{SS} - B u_s \partial_S - u_s \partial_q)J(t, S, q)  \Big\} = 0 \text{ on $[0, T] \times (0, \infty)\times [0, N]$}
\\
J(T, S, q) = Sq - R q^2 \text{ $\forall x \in \mathbb{R}$}
\end{cases},
$$
where $R$ is the terminal cost.

In [ ]:
class LIQUIDATION(HBJ):
    def __init__(self):
        super().__init__()
        self.σ = 0.3
        self.D = 0.05
        self.C = 0.1
        self.R = self.D + self.C
        self.B = 0.0001
        
        
        self.var_dim_J = 4 # (S, q, X, t)
        self.control_vars = [0, 1, 2, 3] # (S ,q, X, t)
        self.cost_function = lambda u, var: 0
        self.differential_operator = lambda J, u, var: 0.5*self.σ**2*Δ(J, var[0]) -self.B*u*div(J, var[0]) - u*div(J, var[1]) + (var[0]*u - self.C*var[1]**2 - self.D*u**2)*div(J, var[2])
        self.domain_func = [(lambda var: [(var[0]-0.5)*4, (var[1]-0.4)*4, (var[2]-0.5)*4, var[3]], 64)]
        self.boundary_cond_J = [lambda J, var: J - (var[2] + var[0]*var[1]-self.R*var[1]**2)]
        self.boundary_func_J = [(lambda var: [(var[0]-0.5)*4, (var[1]-0.4)*4, (var[2]-0.5)*4, var[3]*0 + 1], 64)]

In [ ]:
LIQUIDATION_MODEL_CONFIG = {
    "hidden_dim": 32,
    "learning_rate": 5e-3,
    "loss_weights": (1, 1),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "FF",
    "optimiser": "Adam",
    "delay_control": 2,
    "alpha_noise": 0.3
}
eq = LIQUIDATION()
model = LIQUIDATION_MODEL_CONFIG
solver = DGMPIASolver(model, eq)
loss = np.array(list(solver.train(500)))
plot_losses(loss)

In [ ]:
n = 50
c_xs = np.zeros(n)
c_Ss = np.zeros(n)
c_Q = np.zeros(n)
c_xs[0] = 0.3
c_Ss[0] = 0.5
c_Q[0] = 1.0
uc_xs = np.zeros(n)
uc_Ss = np.zeros(n)
uc_xs[0] = c_xs[0]
uc_Ss[0] = c_Ss[0]
dt = 1/n
ts = [t for t in np.linspace(0, 1, n)]
c_value = np.zeros(n)
for i in range(n-1):
    dW = np.sqrt(dt)*np.random.randn()
    c = solver.u(c_Ss[i], c_Q[i], c_xs[i], i*dt).item()
    c_value[i] = solver.J(c_Ss[i], c_Q[i], c_xs[i], i*dt).item()
    c_Q[i+1] = c_Q[i] - c*dt
    c_Ss[i+1] = c_Ss[i] -eq.B*c*dt + eq.σ*dW
    uc_Ss[i+1] = uc_Ss[i] + eq.σ*dW
    c_xs[i+1] = c_xs[i] + c_Ss[i]*c*dt - eq.D*c**2*dt - eq.C*c_Q[i]**2*dt
    uc_xs[i+1] = uc_xs[i] - eq.C*c_Q[0]**2*dt

c_value[-1] = solver.J(c_Ss[-1], c_Q[-1], c_xs[-1], n*dt).item()
c_xs[-1] += c_Ss[-1]*c_Q[-1] - eq.R*c_Q[-1]**2
uc_xs[-1] += uc_Ss[-1]*c_Q[0] - eq.R*c_Q[0]**2
    
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=ts, y=c_xs, mode='lines', name="Controlled", line=dict(color="#00e476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=uc_xs, mode='lines', name="Uncontrolled", line=dict(color="#FFe476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=c_Ss, mode='lines', name="Value Function", line=dict(color="#eeee76")), row=1, col=1)
fig.update_layout(
    title="Maximise amount of X",
    xaxis_title="t",
    yaxis_title="X",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

### Linear-quadratic control problem 1-dimensional (Riccati Equation) 

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_sX_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

We aim to maximise
$$
J^u(t, x) := \mathbb{E}^{t, x} \Big[ \int_t^T X_s^T C_s X_s + u_s^T D_s u_s ds + X_T^T R X_T\Big],
$$
with $C(t) = C \leq 0, R \leq 0$, and $D=D(t) < -\delta < 0$ given and deterministic ($\delta > 0$ some constant).

We write down the problem in its primal form as
$$
\begin{cases}
\partial_t J(t, x) + \sup_{u} \Big\{ \frac{1}{2} \sigma^2 \partial_{xx} J(t, x) + [H x + M u] \partial_x J(t, x) + C x^2 + D u^2 \Big\} = 0 \text{ on $[0, T] \times (-\infty, \infty)$}
\\
J(T, x) = Rx^2 \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

#### Analytical Solution (Oksendal):

As $J(T, x) = Rx^2$, we assume the form $J(T, x) = S(t) x^2 + b(t)$ for some differentibale $S$ and $b$. The problem can be reformulated as
$$
\begin{cases}
S'(t)x^2 + b'(t) + \sigma^2 S(t) + 2HS(t)x^2 + C x^2 + \sup_{u} \Big\{ 2MS(t) u x + D u^2 \Big\} = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
S(T) = R, b(t)=0 \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

We see that for fixed $t$ and $x$, we have $u^*=u^*(t, x)=-D^{-1} MS(t) x$. As a consequence, we can transform the problem to be gives as
$$
\begin{cases}
[S'(t) + 2HS(t) + C - D^{-1} M^2S^2(t)]x^2 + b'(t) + \sigma^2 S(t) = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
S(T) = R, b(t)=0 \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

The problem is satisfied if
$$
S'(t)= D^{-1} M^2S^2(t) - C - 2HS(t), \; S(T)=R \; \text{(Riccati Equation)}
$$
$$
b'(t)= -\sigma^2 S(t), \; b(T)=0
$$

It follows that $b(t)=  \sigma^2 \int_t^T S(r) dr$, while the Riccati Equation has a unique solution.

In [ ]:
class LQR(HBJ):
    def __init__(self):
        super().__init__()
        self.H = 0.1
        self.M = 2.0
        self.C = -2.0
        self.R = -0.001
        self.D = -0.2
        self.σ = 0.3
        
        self.var_dim_J = 2 # (x, t)
        self.control_vars = [0, 1]
        self.cost_function = lambda u, var: self.C*var[0]**2 + self.D*u**2
        self.differential_operator = lambda J, u, var: (self.H*var[0] + self.M*u)*div(J, var[0]) + (1/2)*self.σ**2*Δ(J, var[0])
        self.domain_func = [(lambda var: [var[0]*2-1, var[1]], 128)]
        self.boundary_cond_J = [lambda J, var: J - self.R*var[0]**2]
        self.boundary_func_J = [(lambda var: [var[0]*2-1, var[1]*0 + 1], 64)]
        self.boundary_cond_u = [lambda u, var: torch.clamp(div(u, var[0]), min=0)]
        self.boundary_func_u = [(lambda var: [var[0]*2-1, var[1]], 64)]

In [ ]:
LQR_MODEL_CONFIG = {
    "hidden_dim": 64,
    "learning_rate": 5e-3,
    "loss_weights": (1, 1),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "GRU",
    "optimiser": "Adam",
    "delay_control": 2,
    "alpha_noise": 0.1
}
eq = LQR()
model = LQR_MODEL_CONFIG
solver = DGMPIASolver(model, eq)
loss = np.array(list(solver.train(400)))
plot_losses(loss)

#### Control function approximation:

In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
xs = np.linspace(-1, 1, 100)
ts = np.linspace(0, 1, 100)
us_pred = np.array([[solver.u(x, t).item() for x in xs] for t in ts])
fig.add_trace(go.Surface(x=xs, y=ts, z=us_pred), row=1, col=1)
fig.update_layout(title='Approximation',
                  scene = dict(
                      xaxis_title="x",
                      yaxis_title="t",
                      zaxis_title="u(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
fig.show()

#### Value function approximation:

In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
xs = np.linspace(-1, 1, 100)
ts = np.linspace(0, 1, 100)
us_pred = np.array([[solver.J(x, t).item() for x in xs] for t in ts])
fig.add_trace(go.Surface(x=xs, y=ts, z=us_pred), row=1, col=1)
fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="x",
                    yaxis_title="t",
                    zaxis_title="J(x, t"),
                  scene2 = dict(
                    xaxis_title="x",
                    yaxis_title="t",
                    zaxis_title="J(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
fig.show()

#### Simulating the Process:

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_s X_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

In [ ]:
n = 50
c_xs = np.zeros(n)
c_xs[0] = 0.75
uc_xs = np.zeros(n)
uc_xs[0] = c_xs[0]
dt = 1/n
ts = [t for t in np.linspace(0, 1, n)]
c_cum_cost = np.zeros(n)
uc_cum_cost = np.zeros(n)
for i in range(n-1):
    dW = np.sqrt(dt)*np.random.randn()
    c = solver.u(c_xs[i], i*dt).item() #-(1/eq.D)*eq.M*((0.316228*np.exp(12.6491*i*dt) - 99125.6)/(313463 + np.exp(12.6491*i*dt))) *c_xs[i] 
    uc = 0
    c_xs[i+1] = c_xs[i] + (eq.H*c_xs[i] + eq.M*c)*dt + eq.σ*dW
    uc_xs[i+1] = uc_xs[i] + (eq.H*uc_xs[i] + eq.M*uc)*dt + eq.σ*dW
    c_cum_cost[i+1] = c_cum_cost[i] + eq.C*c_xs[i]**2 + eq.D*c**2
    uc_cum_cost[i+1] = uc_cum_cost[i] + eq.C*uc_xs[i]**2 + eq.D*uc**2

c_cum_cost[-1] += eq.R*c_xs[-1]**2
uc_cum_cost[-1] += eq.R*uc_xs[-1]**2
    
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=ts, y=c_xs, mode='lines', name="Controlled", line=dict(color="#00e476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=uc_xs, mode='lines', name="Uncontrolled", line=dict(color="#FFe476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=c_cum_cost, mode='lines', showlegend=False, line=dict(color="#00e476")), row=1, col=2)
fig.add_trace(go.Scatter(x=ts, y=uc_cum_cost, mode='lines', showlegend=False, line=dict(color="#FFe476")), row=1, col=2)
fig.update_layout(
    title="Minimise amount of X | Minimise the costs (hold both close to zero)",
    xaxis_title="t",
    yaxis_title="X",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

Example: Assume that the UK is subject to another disease outbreak. Each patient admission costs certain resources. We intend to reduce the amount of people in hospitals as quickly as possible to bound admission costs.

## !!! Problems: For high terminal cost, the value function approximation becomes inaccurate. For 0 terminal cost the control solution becomes a divergingly high control signal

## !!! Any value of H positive leads to a control solution that diverge, taking the negative of that control signal seems optimal

## !!! This is because the control function exploits erroneous approximations of the value function

==> Counteract by introducing regularizing noise to control function and delaying the control update

### Linear-quadratic control problem N-dimensional (Riccati Equation) 

We consider the same problem with $H_t \in \mathbb{R}^{n \times n}$,
$M_t \in \mathbb{R}^{n \times k}$, $\sigma_t \in \mathbb{R}^{n \times m}$ (for now a scalar), $C_t \in \mathbb{R}^{n \times n}$, $D_t \in \mathbb{R}^{k \times n}$, and $R \in \mathbb{R}^{n \times n}$. We also have, $u(t, X_t) \in\mathbb{R}^k$.

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_sX_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

We aim to maximise
$$
J^u(t, x) := \mathbb{E}^{t, x} \Big[ \int_t^T X_s^T C_s X_s + u_s^T D_s u_s ds + X_T^T R X_T\Big],
$$
with $C(t) = C \leq 0, R \leq 0$, and $D=D(t) < -\delta < 0$ given and deterministic ($\delta > 0$ some constant).

We write down the problem in its primal form as
$$
\begin{cases}
\partial_t J(t, x) + \sup_{u} \Big\{ \frac{1}{2} \sigma^2 \sum_{1 \leq i, j \leq n} \Delta_{x_i x_j} J(t, x) + [H x + M u]^T \cdot \nabla_x J(t, x) + x^T C x + u^T D u \Big\} = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
J(T, x) = x^T R x \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

For now, we will consider the problem without the terminal condition.

In [ ]:
class LQR_N(HBJ):
    def __init__(self):
        super().__init__()
        self.H = torch.tensor([[0.1, 0],
                              [0.0, 0.1]])
        self.M = torch.tensor([[2.0, 0],
                              [0.0, 2.0]])
        self.C = torch.tensor([[-2.0, 0],
                              [0.0, -2.0]])
        self.R = torch.tensor([[-0.001, 0],
                              [0.0, -0.001]])
        self.D = torch.tensor([[-0.2, 0],
                              [0.0, -0.2]])
        self.σ = 0.1
        
        self.var_dim_J = 3 # (x, y, t)
        self.sol_dim = 2
        self.control_vars = [0, 1, 2]
        self.cost_function = lambda u, var: (var[:2] |bdotb| (self.C |mdotb| var[:2])) + ((u |bdotm| self.D) |bdotb| u)
        self.differential_operator = lambda J, u, var: (((self.H |mdotb| var[:2]) + (self.M |mdotb| u)) |bdotb| D(J, var[:2])) + (1/2)*self.σ**2*sum([div(div(J, var[i]), var[j]) for i, j in product(range(2), range(2))])
        self.domain_func = [(lambda var: [var[0]*2-1, var[1]*2-1, var[2]], 128)]
        self.boundary_cond_J = [lambda J, var: J - ((var[:2] |bdotm| self.R) |bdotb| var[:2])]
        self.boundary_func_J = [(lambda var: [var[0]*2-1, var[1]*2-1, var[2]*0 + 1], 64)]
        self.boundary_cond_u = [lambda u, var: torch.clamp(div(u, var[0]), min=0),
                               lambda u, var: torch.clamp(div(u, var[1]), min=0)]
        self.boundary_func_u = [(lambda var: [var[0]*2-1, var[1]*2-1, var[2]], 32),
                               (lambda var: [var[0]*2-1, var[1]*2-1, var[2]], 32)]

In [ ]:
LQR_N_MODEL_CONFIG = {
    "hidden_dim": 64,
    "learning_rate": 5e-3,
    "loss_weights": (1, 1),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "GRU",
    "optimiser": "Adam",
    "delay_control": 2,
    "alpha_noise": 0.1
}
eq = LQR_N()
model = LQR_N_MODEL_CONFIG
solver = DGMPIASolver(model, eq)
loss = np.array(list(solver.train(500)))
plot_losses(loss)

#### Simulating the Process:

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_sX_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

In [ ]:
n = 50
c_xs = np.zeros((2, n))
c_xs[:, 0] = 0.5
uc_xs = np.zeros((2, n))
uc_xs[:, 0] = c_xs[:, 0]
dt = 1/n
ts = [t for t in np.linspace(0, 1, n)]
c_cum_cost = np.zeros(n)
uc_cum_cost = np.zeros(n)
for i in range(n-1):
    dW = np.sqrt(dt)*np.random.randn(2, 1)
    c = np.expand_dims(solver.u(c_xs[0, i], c_xs[1, i], i*dt), axis=1)
    uc = np.zeros((2, 1))
    c_xs[:, None, i+1] = c_xs[:, None, i] + (eq.H.numpy() @ c_xs[:, None, i] + eq.M.numpy() @ c)*dt + eq.σ*dW
    uc_xs[:, None, i+1] = uc_xs[:, None, i] + (eq.H.numpy() @ uc_xs[:, None, i] + eq.M.numpy() @ uc)*dt + eq.σ*dW
    c_cum_cost[i+1] = c_cum_cost[i] + c_xs[:, None, i].T @ eq.C.numpy() @ c_xs[:, None, i] + c.T @ eq.D.numpy() @ c
    uc_cum_cost[i+1] = uc_cum_cost[i] + uc_xs[:, None, i].T @ eq.C.numpy() @ uc_xs[:, None, i] + uc.T @ eq.D.numpy() @ uc

c_cum_cost[-1] += c_xs[:, None, -1].T @ eq.R.numpy() @ c_xs[:, None, -1]
uc_cum_cost[-1] += uc_xs[:, None, -1].T @ eq.R.numpy() @ uc_xs[:, None, -1]
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=ts, y=c_xs[0], mode='lines', name="Controlled", line=dict(color="#00e476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=c_xs[1], mode='lines', showlegend=False, line=dict(color="#00e476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=uc_xs[0], mode='lines', name="Uncontrolled", line=dict(color="#FFe476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=uc_xs[1], mode='lines', showlegend=False, line=dict(color="#FFe476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=c_cum_cost, mode='lines', showlegend=False, line=dict(color="#00e476")), row=1, col=2)
fig.add_trace(go.Scatter(x=ts, y=uc_cum_cost, mode='lines', showlegend=False, line=dict(color="#FFe476")), row=1, col=2)
fig.update_layout(
    title="Minimise amount of X | Minimise the costs (hold both close to zero)",
    xaxis_title="t",
    yaxis_title="X",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

### Black-Scholes-Barenblatt

$$
\begin{cases}
\partial_t J(t, x) + \frac{1}{2} \text{trace}(\sigma^2 \text{diag}(x^2 )\Delta J(t, x)) + r(\nabla^T J X - J) = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
J(T, x) = x^2 \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

In [ ]:
CONFIG_BSB = {
    "hidden_dim": 64,
    "learning_rate": 5e-3,
    "loss_weights": (1, 1),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "GRU",
    "optimiser": "Adam",
    "method": "Galerkin"
}
class BSB(PDE):
    def __init__(self):
        super().__init__()
        r = 0.05
        sigma = 0.3
        self.var_dim = 2 # var = (x, t)
        self.equation = lambda u, var: div(u, var[-1]) + (1/2)*sum([X**2*Δ(u, var[:-1]) for X in var[:-1]]) + r*((D(u, var[:-1])|bdotb| var[:-1])- u)
        self.domain_func = [(lambda var: [(x-0.5)*4 for x in var[:-1]]+ [var[-1]], 128)]
        self.boundary_cond = [lambda u, var: u - (var[:-1] |bdotb| var[:-1])]
        self.boundary_func = [(lambda var: [(x-0.5)*4 for x in var[:-1]]+ [var[-1]*0+1], 64)]

In [ ]:
eq = BSB()
model = CONFIG_BSB
solver = DeepPDESolver(model, eq)
losses = list(solver.train(800))

In [ ]:
avgs = np.convolve(losses, np.ones(10), 'valid') / 10
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=np.arange(len(avgs)), y=avgs, mode='lines', name="Error at x=0.1"), row=1, col=1)
fig.update_layout(
    title="Loss",
    xaxis_title="Iterations",
    yaxis_title="Loss",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

In [ ]:
from FBSDEs import FBSDESolver
from pdes import FBSDE
class BSB(FBSDE):
    def __init__(self):
        super().__init__()
        r = 0.05
        self.h = lambda X, Y, Z, t: r*(Y-torch.einsum("bi, bi -> b", Z, X).unsqueeze(1))
        
        self.b = lambda X, t: 0.0*X
        self.sigma = lambda X, t: 0.3*torch.diag_embed(X)
        
        self.terminal_condition = lambda X: torch.einsum("bi, bi-> b", X, X).unsqueeze(1)
        
        self.var_dim = 2
        self.terminal_time = 1     
        self.init_sampling_func = lambda X: (X-0.5)*2
        self.control_noise = 0.0

In [ ]:
BSB_MODEL_CONFIG = {
    "batch_size": 128,
    "num_discretisation_steps": 30,
    "hidden_dim": 64,
    "learning_rate": 5e-3,
    "lr_decay": 0.99,
    "network_type": "MINI",
    "optimiser": "Adam"
}
eq_sim = BSB()
solver_sim = FBSDESolver(BSB_MODEL_CONFIG, eq_sim)

In [ ]:
num_samples = 2
def J_sol(X, t):
    r = 0.05
    sigma = 0.3
    return np.exp((r + sigma**2)*(1 - t))*np.sum(X**2, axis=-1, keepdims=True)
Xs, _, ts = solver_sim.simulate_processes(num_samples)
Y_sol = J_sol(Xs, ts)

In [ ]:
Y_preds = np.array([[solver.u(*x_b) for x_b in x] for x in Xs])

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=ts[:, 0].flatten(), y=Y_preds[:, 0].flatten(), mode='lines', name="Prediction", line=dict(color="#FFe476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts[:, 0].flatten(), y=Y_sol[:, 0].flatten(), mode='lines', name="Ground truth", line=dict(color="#00e476")), row=1, col=1)
for i in range(1, num_samples):
    fig.add_trace(go.Scatter(x=ts[:, i].flatten(), y=Y_preds[:, i].flatten(), mode='lines', showlegend=False, line=dict(color="#FFe476")), row=1, col=1)
    fig.add_trace(go.Scatter(x=ts[:, i].flatten(), y=Y_sol[:, i].flatten(), mode='lines', showlegend=False, line=dict(color="#00e476")), row=1, col=1)
fig.update_layout(
    title="Loss",
    xaxis_title="t",
    yaxis_title="J",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

### Allen-Cahn

In [101]:
CONFIG_AC = {
    "hidden_dim": 128,
    "learning_rate": 5e-3,
    "loss_weights": (1, 5),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "RES",
    "optimiser": "Adam",
    "method": "Galerkin"
}
class AC(PDE):
    def __init__(self):
        super().__init__()
        self.d = 1.85
        self.var_dim = 20 + 1# var = (x, t)
        self.equation = lambda u, var: div(u, var[-1]) + (1/2)*Δ(u, var[:-1]) + u - u**3
        self.domain_func = [(lambda var: [(x-0.5)*self.d for x in var[:-1]]+ [var[-1]*0.3], 128)]
        self.boundary_cond = [lambda u, var: u - 1/(2+0.4*torch.sum(cat(var[:-1])**2, dim=-1, keepdims=True))]#,
                             #lambda u, var: torch.sum(D(u, var[:-1])*self.normal, dim=-1)]
        self.boundary_func = [(lambda var: [(x-0.5)*self.d for x in var[:-1]]+ [var[-1]*0 + 0.3], 64)]#,
                             #(lambda var: self.sample_boundary(var[:-1]) + [var[-1]*0.3], 64)]
        #self.normal = torch.zeros((self.boundary_func[1][1], self.var_dim-1))
        
    def sample_boundary(self, var):
        is_boundary = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], 1)) # idxs for var_dim
        mask = F.one_hot(is_boundary, num_classes=self.var_dim-1).squeeze(1)
        not_mask = -(mask-1)
        sign = torch.sign(torch.rand(size=(var[-1].shape[0], self.var_dim-1))-0.5) # integers -1, 1
        re = [(v-0.5)*self.d*not_mask[:, None, i] + mask[:, None, i]*sign[:, None, i] for i, v in enumerate(var)]
        self.normal = mask
        return re
        
        

In [102]:
eq = AC()
model = CONFIG_AC
solver = DeepPDESolver(model, eq)
losses = list(solver.train(400))
plot_loss(losses)

100%|██████████| 400/400 [02:38<00:00,  2.52 it/s]


In [103]:
from FBSDEs import FBSDESolver
from pdes import FBSDE
AC_MODEL_CONFIG = {
    "batch_size": 128,
    "num_discretisation_steps": 15,
    "hidden_dim": 128,
    "learning_rate": 5e-3,
    "lr_decay": 0.99,
    "network_type": "MINI",
    "optimiser": "Adam"
}
model = AC_MODEL_CONFIG
class AC(FBSDE):
    def __init__(self):
        super().__init__()
        self.h = lambda X, Y, Z, t: Y - Y**3
        
        self.b = lambda X, t: 0.0*X
        self.sigma = lambda X, t: torch.diag_embed(X*0+1)
        
        self.terminal_condition = lambda X: 1/(2+0.4*torch.sum(X**2, dim=-1, keepdims=True))
        
        self.var_dim = 20
        self.terminal_time = 0.3   
        self.init_sampling_func = lambda X: X*0
        self.control_noise = 0.5

In [104]:
eq_2 = AC()
solver_2 = FBSDESolver(model, eq_2)
num_samples=5
Xs, _, ts = solver_2.simulate_processes(num_samples)
final_test_values = 1/(2+0.4*np.sum(Xs[-1]**2, axis=-1, keepdims=True))

In [105]:
ts = np.linspace(0, 0.3, 15)
fig = make_subplots(rows=1, cols=1)
for j in range(0, num_samples):
    xs = np.array([solver.u(*(Xs[i, j]), ts[i]).item() for i in range(15)])
    fig.add_trace(go.Scatter(x=ts, y=xs, mode='lines', showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[0.3]*num_samples, y=final_test_values.flatten(), mode='markers', showlegend=False), row=1, col=1)
fig.update_layout(
    title="Allen-Cahn",
    xaxis_title="Iterations",
    yaxis_title="Loss",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()